# MLP Training on Extracted Features

This notebook loads precomputed feature vectors and trains a small MLP classifier. It tracks metrics and saves the best model.

In [1]:
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Paths
repo_root = Path('..').resolve()
feature_dir = Path('.')

train_feat_path = feature_dir / 'train_features.npy'
val_feat_path = feature_dir / 'val_features.npy'
train_lbl_path = feature_dir / 'train_labels.npy'
val_lbl_path = feature_dir / 'val_labels.npy'

for p in [train_feat_path, val_feat_path, train_lbl_path, val_lbl_path]:
    if not p.exists():
        raise FileNotFoundError(f'Missing file: {p.resolve()}')

print('Feature files found.')

Feature files found.


In [3]:
# Load features and labels
train_features = np.load(train_feat_path)
val_features = np.load(val_feat_path)
train_labels = np.load(train_lbl_path)
val_labels = np.load(val_lbl_path)

print('Train features:', train_features.shape)
print('Val features:', val_features.shape)
print('Train labels:', train_labels.shape)
print('Val labels:', val_labels.shape)

Train features: (5216, 512)
Val features: (16, 512)
Train labels: (5216,)
Val labels: (16,)


In [4]:
# Dataloaders
batch_size = 64
train_ds = TensorDataset(
    torch.from_numpy(train_features).float(),
    torch.from_numpy(train_labels).long()
)
val_ds = TensorDataset(
    torch.from_numpy(val_features).float(),
    torch.from_numpy(val_labels).long()
)

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

print(f'Train batches: {len(train_loader)} | Val batches: {len(val_loader)}')

Train batches: 82 | Val batches: 1


In [5]:
# MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_dim=512, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.2),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        return self.net(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MLPClassifier(input_dim=train_features.shape[1]).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

print(model)
print('Device:', device)

MLPClassifier(
  (net): Sequential(
    (0): Linear(in_features=512, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=128, out_features=2, bias=True)
  )
)
Device: cpu


In [6]:
# Training loop with metric tracking
def evaluate(model, loader):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            logits = model(x)
            loss = criterion(logits, y)
            total_loss += loss.item() * x.size(0)
            preds = torch.argmax(logits, dim=1)
            correct += (preds == y).sum().item()
            total += x.size(0)
    avg_loss = total_loss / total
    acc = correct / total
    return avg_loss, acc

num_epochs = 20
best_val_loss = float('inf')
best_path = Path('best_mlp_model.pt')

history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)
        preds = torch.argmax(logits, dim=1)
        correct += (preds == y).sum().item()
        total += x.size(0)

    train_loss = running_loss / total
    train_acc = correct / total
    val_loss, val_acc = evaluate(model, val_loader)

    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f'Epoch {epoch:02d} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'input_dim': train_features.shape[1],
            'num_classes': 2
        }, best_path)
        print(f'Saved best model to {best_path.resolve()}')

Epoch 01 | Train Loss: 0.6000 Acc: 0.7402 | Val Loss: 0.8829 Acc: 0.5000
Saved best model to /Users/meetmehta/Desktop/project/DL Project/pneumonia-detection/notebooks/best_mlp_model.pt
Epoch 02 | Train Loss: 0.5872 Acc: 0.7429 | Val Loss: 0.8029 Acc: 0.5000
Saved best model to /Users/meetmehta/Desktop/project/DL Project/pneumonia-detection/notebooks/best_mlp_model.pt
Epoch 03 | Train Loss: 0.5598 Acc: 0.7429 | Val Loss: 0.7443 Acc: 0.5000
Saved best model to /Users/meetmehta/Desktop/project/DL Project/pneumonia-detection/notebooks/best_mlp_model.pt
Epoch 04 | Train Loss: 0.4796 Acc: 0.7692 | Val Loss: 0.5854 Acc: 0.5625
Saved best model to /Users/meetmehta/Desktop/project/DL Project/pneumonia-detection/notebooks/best_mlp_model.pt
Epoch 05 | Train Loss: 0.4186 Acc: 0.8160 | Val Loss: 1.0973 Acc: 0.5625
Epoch 06 | Train Loss: 0.3678 Acc: 0.8422 | Val Loss: 1.1265 Acc: 0.5625
Epoch 07 | Train Loss: 0.3651 Acc: 0.8395 | Val Loss: 0.8203 Acc: 0.5625
Epoch 08 | Train Loss: 0.3724 Acc: 0.8370